# Agentic retrieval using Azure AI Search Index Knowledge Source

Use this notebook to create an agentic retrieval pipeline from an existing Azure AI Search index.

In this walkthrough, you will:

+ Create a search index with semantic configuration
+ Upload sample documents to the index
+ Create a knowledge source that points to the search index
+ Create a knowledge base for intelligent query planning
+ Create a Foundry agent to determine when queries are needed
+ Create an MCP Tool Connection for authentication
+ Start a chat with the agent

This notebook demonstrates how to leverage existing search indexes without re-ingestion.

## Prerequisites

+ Azure AI Search, basic tier or higher, in [any region that supports semantic ranker](https://learn.microsoft.com/azure/search/search-region-support#azure-public-regions).

+ Azure OpenAI, and you should have an **Azure AI Developer** role assignment to create a Foundry project.

+ An [Azure AI agent and Foundry project](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal), created in the Azure AI Foundry portal, with the basic setup, used for creating the Foundry agent.

+ A deployment of a [supported model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) in your Foundry project. This notebook uses gpt-4o-mini. We recommend 100,000 token capacity.

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment.

## Install required packages

Install the Azure SDK packages needed for this notebook.

In [ ]:
%pip install azure-search-documents==11.7.0b2
%pip install azure-identity
%pip install azure-ai-projects
%pip install azure-mgmt-cognitiveservices
%pip install python-dotenv
%pip install requests

## Load connections

Load the environment variables to set up connections and object names.

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.mgmt.core.tools import parse_resource_id
import os

load_dotenv(override=True)

project_endpoint = os.environ["PROJECT_ENDPOINT"]
project_resource_id = os.environ["PROJECT_RESOURCE_ID"]
project_connection_name = os.getenv("PROJECT_CONNECTION_NAME", "productsknowledgeconnection")
agent_model = os.getenv("AGENT_MODEL", "gpt-4o-mini")
agent_name = os.getenv("AGENT_NAME", "products-knowledge-agent")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
search_api_key = os.environ["AZURE_SEARCH_API_KEY"]
credential = DefaultAzureCredential()
azure_key_credential = AzureKeyCredential(search_api_key)
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE_NAME", "products-index-source")
index_name = os.getenv("AZURE_SEARCH_INDEX", "products-index")
base_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "products-knowledge-base")

parsed_resource_id = parse_resource_id(project_resource_id)
subscription_id = parsed_resource_id['subscription']
resource_group = parsed_resource_id['resource_group']
account_name = parsed_resource_id['name']
project_name = parsed_resource_id['child_name_1']

## Create search index

Create a search index with semantic configuration for better search results.

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField
)
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True),
        SearchField(name="title", type="Edm.String", searchable=True, filterable=False, sortable=False),
        SearchField(name="content", type="Edm.String", searchable=True, filterable=False, sortable=False),
        SearchField(name="category", type="Edm.String", searchable=False, filterable=True, sortable=True, facetable=True),
        SearchField(name="price", type="Edm.Double", searchable=False, filterable=True, sortable=True, facetable=True),
        SearchField(name="inStock", type="Edm.Boolean", searchable=False, filterable=True, sortable=False),
        SearchField(name="tags", type="Collection(Edm.String)", searchable=True, filterable=True, facetable=True)
    ],
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    title_field=SemanticField(field_name="title"),
                    content_fields=[SemanticField(field_name="content")],
                    keywords_fields=[SemanticField(field_name="tags")]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")

## Upload sample documents

Upload product documents to the search index.

In [ ]:
from azure.search.documents import SearchClient

documents = [
    {
        "id": "1",
        "title": "Premium Laptop Pro 15",
        "content": "High-performance laptop featuring 15-inch 4K display, Intel Core i9 processor, 32GB DDR5 RAM, 1TB NVMe SSD, NVIDIA RTX 4060 graphics. Perfect for content creators, developers, and power users. Includes Thunderbolt 4 ports and Wi-Fi 6E.",
        "category": "electronics",
        "price": 2499.99,
        "inStock": True,
        "tags": ["laptop", "high-performance", "gaming", "professional"]
    },
    {
        "id": "2",
        "title": "Wireless Ergonomic Mouse",
        "content": "Ergonomic wireless mouse designed for all-day comfort. Features precision optical sensor with adjustable DPI (800-3200), 6 programmable buttons, and rechargeable battery with 60-day battery life. Compatible with Windows, Mac, and Linux.",
        "category": "accessories",
        "price": 79.99,
        "inStock": True,
        "tags": ["mouse", "wireless", "ergonomic", "productivity"]
    },
    {
        "id": "3",
        "title": "USB-C Docking Station Pro",
        "content": "Professional USB-C docking station with dual 4K monitor support, 100W power delivery, Gigabit Ethernet, SD card reader, and multiple USB 3.2 ports. Ideal for home office and hybrid work setups. Supports Windows, Mac, and Chrome OS.",
        "category": "accessories",
        "price": 249.99,
        "inStock": False,
        "tags": ["dock", "usb-c", "dual-monitor", "workstation"]
    },
    {
        "id": "4",
        "title": "4K Webcam with AI Features",
        "content": "Professional 4K webcam with AI-powered auto-framing, background blur, and low-light correction. Built-in dual microphones with noise cancellation. Perfect for video conferencing, streaming, and content creation. Plug-and-play USB connection.",
        "category": "electronics",
        "price": 199.99,
        "inStock": True,
        "tags": ["webcam", "4k", "video-conferencing", "streaming"]
    },
    {
        "id": "5",
        "title": "Mechanical Keyboard RGB",
        "content": "Premium mechanical keyboard with Cherry MX switches, customizable RGB backlighting, programmable macro keys, and detachable USB-C cable. Features aluminum frame, dedicated media controls, and included wrist rest. Available in multiple switch types.",
        "category": "accessories",
        "price": 159.99,
        "inStock": True,
        "tags": ["keyboard", "mechanical", "rgb", "gaming", "typing"]
    },
    {
        "id": "6",
        "title": "27-inch 4K Monitor",
        "content": "Professional 27-inch 4K IPS monitor with 99% sRGB color accuracy, HDR400, 60Hz refresh rate, and USB-C connectivity with 65W power delivery. Includes height-adjustable stand and VESA mount compatibility. Ideal for creative professionals.",
        "category": "electronics",
        "price": 449.99,
        "inStock": True,
        "tags": ["monitor", "4k", "display", "professional"]
    }
]

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents=documents)
print(f"Uploaded {len(documents)} documents to index '{index_name}'")

## Create a knowledge source

Create a knowledge source that targets the existing search index.

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndexKnowledgeSource,
    SearchIndexKnowledgeSourceParameters,
    SearchIndexFieldReference
)

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for product catalog",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_fields=[
            SearchIndexFieldReference(name="id"),
            SearchIndexFieldReference(name="category"),
            SearchIndexFieldReference(name="price"),
            SearchIndexFieldReference(name="inStock")
        ]
    ),
)

index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

## Create a knowledge base

Create a knowledge base that wraps the knowledge source.

`EXTRACTIVE_DATA` is the default modality and returns content from your knowledge sources without generative alteration. This is recommended for interaction with Foundry Agent Service.

In [ ]:
from azure.search.documents.indexes.models import (
    KnowledgeBase,
    KnowledgeSourceReference,
    KnowledgeRetrievalOutputMode,
    KnowledgeRetrievalMinimalReasoningEffort
)

knowledge_base = KnowledgeBase(
    name=base_name,
    knowledge_sources=[
        KnowledgeSourceReference(name=knowledge_source_name)
    ],
    output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA,
    retrieval_reasoning_effort=KnowledgeRetrievalMinimalReasoningEffort()
)

index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)
print(f"Knowledge base '{base_name}' created or updated successfully")

mcp_endpoint = f"{endpoint}/knowledgebases/{base_name}/mcp?api-version=2025-11-01-Preview"

## Create an MCP Tool Connection

Create a connection to authenticate to your knowledge base tool.

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.cognitiveservices.models import (
    ConnectionPropertiesV2BasicResource,
    CustomKeysConnectionProperties,
    CustomKeys
)

mgmt_client = CognitiveServicesManagementClient(credential, subscription_id)
resource = mgmt_client.project_connections.create(
    resource_group_name=resource_group,
    account_name=account_name,
    project_name=project_name,
    connection_name=project_connection_name,
    connection=ConnectionPropertiesV2BasicResource(
        properties=CustomKeysConnectionProperties(
            category="RemoteTool",
            target=mcp_endpoint,
            is_shared_to_all=True,
            metadata={"ApiType": "Azure"},
            credentials=CustomKeys(
                keys={"api-key": search_api_key}
            )
        )
    )
)

print(f"Connection '{resource.name}' created or updated successfully.")

## Create an Azure AI Agent

Create an agent that uses the knowledge base to answer questions about products.

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

instructions = """
A Q&A agent that can answer questions about products in our catalog.
Always provide references to the ID and category of the products used to answer the question.
If you do not have the answer, respond with "I don't know".
"""

mcp_kb_tool = MCPTool(
    server_label="knowledge-base",
    server_url=mcp_endpoint,
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id=project_connection_name
)

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=agent_model,
        instructions=instructions,
        tools=[mcp_kb_tool]
    )
)

print(f"AI agent '{agent_name}' created or updated successfully")

## Start a chat with the agent

In [ ]:
openai_client = project_client.get_openai_client()

conversation = openai_client.conversations.create()

response = openai_client.responses.create(
    conversation=conversation.id,
    input="What laptops do you have available and what are their prices?",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"Response: {response.output_text}")

## Query with filters

Ask questions that leverage the filterable fields in the index.

In [ ]:
response = openai_client.responses.create(
    conversation=conversation.id,
    input="Show me electronics under $300 that are in stock",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"Response: {response.output_text}")

## Clean up objects and resources

Delete resources when you're done.

### Delete the agent

In [ ]:
project_client.agents.delete_version(agent.name, agent.version)
print(f"AI agent '{agent.name}' version '{agent.version}' deleted successfully")

### Delete the knowledge base

In [ ]:
index_client.delete_knowledge_base(base_name)
print(f"Knowledge base '{base_name}' deleted successfully")

### Delete the knowledge source

In [ ]:
index_client.delete_knowledge_source(knowledge_source=knowledge_source_name)
print(f"Knowledge source '{knowledge_source_name}' deleted successfully.")

### Delete the index

In [ ]:
index_client.delete_index(index)
print(f"Index '{index_name}' deleted successfully")